爬蟲抓取語言學習 與證照考試app 0119

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from google_play_scraper import app
import re
import emoji
import os
from datetime import datetime
import pytz


google-play-scraper：用於爬取 Google Play 商店的評論與應用程式資訊。

Requests: 透過 Python 做數據分析或網路爬蟲時幾乎一定會上網下載網頁原始碼或其他 HTTP 請求

pandas：方便整理與分析爬取到的評論資料。

numpy：數據運算工具，後續做基礎處理時會用到。

emoji：能將評論中出現的 emoji 做解析與處理，避免存入資料庫時出現亂碼。

appId,title,description,installs,score

function 1：抓 App ID

In [75]:
key_category = ['language learning','Education','vocabulary','English','listen','Grammar','TOEIC','IELTS','ljanpanese','teaching','speak','French','AI learning']

In [76]:
key_appId = []
from google_play_scraper import search
for category in key_category:
    result = search(
    category,
    lang="en",  # defaults to 'en'
    country="us",  # defaults to 'us'
    n_hits=30  # defaults to 30 (= Google's maximum)
    )
    for x in result:
        key_appId.append(x['appId'])

In [24]:
print(key_appId)
print(len(key_appId))


['com.duolingo', 'com.selabs.speak', 'com.busuu.android.enc', 'com.babbel.mobile.android.en', 'com.moymer.falou', 'air.com.rosettastone.mobile.CoursePlayer', 'com.hyperthought.lingolegend', 'com.picoai.languageapp.android', 'com.unacademy.antonio', 'com.linguist', 'com.hellotalk', 'com.memrise.android.memrisecompanion', 'com.simonandschuster.pimsleur.unified.android', 'com.atistudios.mondly.languages', 'com.languagedrops.drops.international', 'com.mango.android', 'es.monkimun.lingokids', 'ai.talkpal', 'com.lingodeer', 'com.pronunciatorllc.bluebird', 'com.david.android.languageswitch', 'io.linga', 'com.innovativelanguage.innovativelanguage101', 'com.preply', 'ai.praktika.android', 'net.tandem', 'com.logicappkids', 'com.ewa.ewaapp', 'com.talkin.learn', 'com.codeway.aitutor', 'com.duolingo', 'org.khankids.android', 'com.education.android.h.intelligence', 'org.khanacademy.android', 'com.duolingo.literacy', 'air.com.starfall.more', 'com.hellosimply.simplydraw', 'com.rvappstudios.abc.spellin

In [77]:
#去重
only = set(key_appId)
print(only)
print(len(only))

{'com.pronunciatorllc.bluebird', 'com.wlingua.curso', 'com.milinix.ieltstest', 'com.lin.duo.hd.french', 'com.verbesconjugaison', 'com.education.android.h.intelligence', 'learn.words.learn.english', 'ai.talkpal', 'com.knudge.me', 'com.wonder', 'com.soundcloud.android', 'tv.englishclub.b2c', 'es.monkimun.lingokids', 'com.uoe.vocabularypro', 'com.prodigygame.prodigy', 'com.learnlangs.learnenglishforbeginnersus', 'com.orangeannoe.englishdictionary', 'com.atistudios.mondly.languages', 'com.eup.migiitoeic', 'com.simonandschuster.pimsleur.unified.android', 'com.jsk.englieshlearning', 'us.nobarriers.elsa', 'com.moymer.falou', 'com.hanum.jinjja', 'com.unacademy.antonio', 'fr.lemonde.ftqmobile', 'com.david.android.languageswitch', 'com.breboucas.francesparaviajar', 'com.masterenglishprogram', 'com.simyasolutions.ling.ja', 'com.hellochinese', 'com.hellosimply.simplydraw', 'com.clearchannel.iheartradio.controller', 'jp.co.latika.android.newtoeicgrammar', 'com.languagedrops.drops.learn.learning.spe

function 2：抓 App 詳細資訊

欄位
appId
title
description
realInstalls
score
Ratings_Count
Reviews_Count

days_since_update

In [78]:
data = []

In [79]:
from google_play_scraper import app
from google_play_scraper.exceptions import NotFoundError
import datetime
from datetime import date
from datetime import datetime

#取得目前時間(年月日)
now = datetime.now()
date_now = now.date()

for id in only:
    try:
        result = app(
        id,
        lang='en', # defaults to 'en'
        country='us' # defaults to 'us'
        )
        #處理時間搓印 -> days_since_update
        if result['updated'] == '' or result['updated'] == None:
            days_since_update = None
        else: 
            seconds_timestamp = int(result['updated'])
            dt_object = datetime.fromtimestamp(seconds_timestamp)
            date_last_updated = dt_object.date()


            delta = date_now - date_last_updated
            days_since_update = delta.days

        #處理survive 這個欄位:days_since_update ≤ 90 → 存活 = 1
        if  days_since_update is None or days_since_update >90 :
            survive = 0
        else:
            survive = 1     

        #rating 有多少人按過星星 用來表示使用者參與情況
        data.append([result['appId'],result['title'],result['description'],result['realInstalls'],result['score'],result['ratings'],result['categories'],days_since_update,survive])
    except NotFoundError:
        print(f"❌ App not found: {id}")
        continue

❌ App not found: None


In [166]:
for x in data:
    print(x)

['com.getmimo', 'Mimo: Learn Coding/Programming', 'Learn coding in Python, JavaScript, HTML, CSS, or SQL, and master modern AI-assisted development to become a developer in the age of AI. Mimo is a coding app that takes you from zero programming skills to building AI-powered projects. Gain the knowledge and tools to create production-ready software and pursue a job as a developer. Start building strong portfolio projects that help you stand out, future-proof your skills, and take the next step beyond just programming syntax. Download Mimo and launch your developer career now!\r\n\r\nWho It\'s For\r\n\r\nThe Mimo programming app stands out by guiding aspiring developers through a proven, structured learning process. Unlike other programming apps that focus on passive tutorials, Mimo delivers hands-on, career-relevant coding skills. You progress from your first line of code in HTML, Python, or any other programming language, to mastering AI-assisted software development, building a portf

In [80]:
df = pd.DataFrame(data)
df.columns = ["appId",
"title",
"description",
"realInstalls",
"score",
"ratings",
"categories",
"days_since_update",
"survive"
]

In [168]:
print(df)

                             appId                           title  \
0                      com.getmimo  Mimo: Learn Coding/Programming   
1    com.nemoapps.android.japanese                   Nemo Japanese   
2               com.hrd.vocabulary  Vocabulary - Learn words daily   
3                    tunein.player    TuneIn Radio: Music & Sports   
4                 de.radio.android   radio.net - AM FM Radio Tuner   
..                             ...                             ...   
249     com.quizlet.quizletandroid   Quizlet: More than Flashcards   
250              com.wlingua.curso         Wlingua - Learn English   
251                  com.engbright  Bright – English for beginners   
252                   com.duolingo      Duolingo: Language Lessons   
253   com.Hello_Hello.English.Main  Learn English with Hello-Hello   

                                           description  realInstalls  \
0    Learn coding in Python, JavaScript, HTML, CSS,...      19673884   
1    Nemo is de

In [169]:
df.to_csv('data.csv', encoding = 'utf-8-sig',index = False)

function 3：抓評論

In [179]:
from google_play_scraper import Sort, reviews
data_review = []
for id in only :
    result, continuation_token = reviews(
        id,
        lang='en', # defaults to 'en'
        country='us', # defaults to 'us'
        sort=Sort.NEWEST, # defaults to Sort.NEWEST / 最新發表的評論
        count=30, # defaults to 100
    )
    for x in result:
        data_review.append([id,x['content']])

# If you pass `continuation_token` as an argument to the reviews function at this point,
# it will crawl the items after 3 review items.



In [ ]:
result, _ = reviews(
    'com.fantome.penguinisle',
    continuation_token=continuation_token ) # defaults to None(load from the beginning)

In [180]:
print(data_review)

[['com.pronunciatorllc.bluebird', 'best for struggling learners and really helps u understand'], ['com.pronunciatorllc.bluebird', "great app, when I'm an adult and have my own money I'll consider getting a subscription. the daily lesson is nice. what you do is incredible, the variety is huge. just please add Yakut/Sakha language? there are no apps that teach it 😞😞😞"], ['com.pronunciatorllc.bluebird', 'My absolute favorite language learning app. The lessons make the words easy to understand and memorize. Never before have I enjoyed trying a new language, especially Turkmen, which I struggled with prior.'], ['com.pronunciatorllc.bluebird', 'this app very useful'], ['com.pronunciatorllc.bluebird', 'This is the best, by far, language learning app that I have ever used.'], ['com.pronunciatorllc.bluebird', "c'est utile pour les débutants comme moi"], ['com.pronunciatorllc.bluebird', 'this is an excellent app to learn what your saying in a new app'], ['com.pronunciatorllc.bluebird', "I love t

In [181]:
df = pd.DataFrame(data_review)
df.columns = ["appId",
"review",
]

In [182]:
df.to_csv('data_reviews.csv', encoding = 'utf-8-sig',index = False)

In [193]:
a = 'df_sorted'
b = a +'clean'
b

'df_sortedclean'

文字前處理

In [188]:
df = pd.read_csv('data_reviews.csv')

In [189]:
print(df)

                             appId  \
0     com.pronunciatorllc.bluebird   
1     com.pronunciatorllc.bluebird   
2     com.pronunciatorllc.bluebird   
3     com.pronunciatorllc.bluebird   
4     com.pronunciatorllc.bluebird   
...                            ...   
7231   io.ivoca.conversationfrench   
7232   io.ivoca.conversationfrench   
7233   io.ivoca.conversationfrench   
7234   io.ivoca.conversationfrench   
7235   io.ivoca.conversationfrench   

                                                 review  
0     best for struggling learners and really helps ...  
1     great app, when I'm an adult and have my own m...  
2     My absolute favorite language learning app. Th...  
3                                  this app very useful  
4     This is the best, by far, language learning ap...  
...                                                 ...  
7231                                           Nice app  
7232                                        Amazing app  
7233                 

clean_review

分隔線

In [190]:
df["review"]
import re
clean_review = []

# 建立 PorterStemmer 物件
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# 在進入迴圈之前，先確保 df["review"] 裡面全部都是字串
df = df.dropna(subset=["review"]) #直接刪除 review 是空的那些資料列

for content in df["review"]:
    text = re.findall(r'[a-z]+',content.lower()) #結果會是list
    stem_token = [ps.stem(t) for t in text]
    stop_words = ['i','it','the','a','an','is','am','in','at','of','on','and','or','to','for','app']
    filter_token = [t for t in stem_token if t not in stop_words and t != '']
    clean_review.append(filter_token)
    

# 新增欄位到原本的df上面
df['clean_review'] = clean_review

    

In [191]:
print(df)

                             appId  \
0     com.pronunciatorllc.bluebird   
1     com.pronunciatorllc.bluebird   
2     com.pronunciatorllc.bluebird   
3     com.pronunciatorllc.bluebird   
4     com.pronunciatorllc.bluebird   
...                            ...   
7231   io.ivoca.conversationfrench   
7232   io.ivoca.conversationfrench   
7233   io.ivoca.conversationfrench   
7234   io.ivoca.conversationfrench   
7235   io.ivoca.conversationfrench   

                                                 review  \
0     best for struggling learners and really helps ...   
1     great app, when I'm an adult and have my own m...   
2     My absolute favorite language learning app. Th...   
3                                  this app very useful   
4     This is the best, by far, language learning ap...   
...                                                 ...   
7231                                           Nice app   
7232                                        Amazing app   
7233        

In [192]:
df.to_csv('data_clean_review.csv', encoding = 'utf-8-sig',index = False)

分割線

tf-idf

針對官方描述

In [4]:
#先嘗試建立字典
df_metadata_clean = pd.read_csv('./data_metadata_clean.csv')

In [5]:
df_metadata_clean['clean_description']
voc = []
for app_desc in df_metadata_clean['clean_description']:
    for term in app_desc.split():
        voc.append(term)

In [6]:
set_voc = set(voc)
print(len(set_voc))
#set_voc : 建立好字典了 是set 格式

4324


In [7]:
list_voc = list(set_voc) #不重複的
print(list_voc)
print(len(list_voc))

['tnw', 'celesti', 'red', 'scroll', 'experienc', 'embed', 'descriptor', 'plural', 'prestigi', 'empathet', 'el', 'disney', 'irish', 'blippi', 'mondli', 'quebec', 'stress', 'hardcor', 'inbox', 'christma', 'inquisit', 'avail', 'old', 'without', 'conson', 'centr', 'p', 'urdu', 'issu', 'backstori', 'gingerproofread', 'ti', 'clearer', 'calcul', 'cq', 'crush', 'fran', 'bing', 'assist', 'instrument', 'ukrain', 'align', 'step', 'anybodi', 'hebrew', 'system', 'chunk', 'unbeat', 'trap', 'by', 'xm', 'advis', 'storybook', 'engag', 'truli', 'bambara', 'pepper', 'trip', 'tweet', 'importantli', 'breakfast', 'mindblow', 'our', 'goal', 'weei', 'hesit', 'sam', 'amaz', 'bookkeep', 'flair', 'housepl', 'southeast', 'nghe', 'school', 'hellosimpli', 'homophon', 'youngest', 'luxembourgish', 'mit', 'embrac', 'korean', 'romanis', 'complimentari', 'mind', 'disclaim', 'handshak', 'reddit', 'rewriteapp', 'scottish', 'voiceov', 'abidjan', 'concert', 'usernam', 'master', 'felt', 'misunderstand', 'mechan', 'subtract',

tf

In [ ]:
#計算tf 使用dataframe
from collections import Counter
all_app_counts = []
#迴圈
for app_desc in df_metadata_clean['clean_description']:
    counts = Counter(app_desc.split()) #每個 App 算好它的詞頻小字典
    total_lengh = len(app_desc.split())
    for k,v in counts.items():
        counts[k] = v/total_lengh
    
    all_app_counts.append(counts)

print(all_app_counts)



[Counter({'talkin': 0.029748283752860413, 'cultur': 0.02745995423340961, 'global': 0.02517162471395881, 'you': 0.02517162471395881, 'real': 0.02288329519450801, 'voic': 0.018306636155606407, 'from': 0.016018306636155607, 'your': 0.016018306636155607, 'connect': 0.013729977116704805, 'peopl': 0.013729977116704805, 'http': 0.013729977116704805, 'com': 0.013729977116704805, 'social': 0.011441647597254004, 'share': 0.011441647597254004, 's': 0.011441647597254004, 'with': 0.011441647597254004, 'no': 0.011441647597254004, 'differ': 0.009153318077803204, 'meet': 0.009153318077803204, 'time': 0.009153318077803204, 'languag': 0.009153318077803204, 'chat': 0.009153318077803204, 'experi': 0.009153318077803204, 'that': 0.009153318077803204, 'world': 0.009153318077803204, 'www': 0.009153318077803204, 'multilingu': 0.006864988558352402, 'commun': 0.006864988558352402, 'parti': 0.006864988558352402, 'smart': 0.006864988558352402, 'match': 0.006864988558352402, 'help': 0.006864988558352402, 'cross': 0

In [9]:
import pandas as pd
#用 Pandas 一口氣對齊所有單字
tf = pd.DataFrame(all_app_counts)
tf = tf.fillna(0)
print(tf)

       talkin    global  multilingu    commun     where      voic      from  \
0    0.029748  0.025172    0.006865  0.006865  0.002288  0.018307  0.016018   
1    0.000000  0.000000    0.000000  0.000000  0.000000  0.000000  0.003226   
2    0.000000  0.000000    0.000000  0.000000  0.002387  0.000000  0.000000   
3    0.000000  0.000000    0.000000  0.000000  0.000000  0.000000  0.007812   
4    0.000000  0.000000    0.000000  0.000000  0.000000  0.000000  0.006579   
..        ...       ...         ...       ...       ...       ...       ...   
250  0.000000  0.000000    0.000000  0.000000  0.000000  0.000000  0.011905   
251  0.000000  0.002079    0.000000  0.006237  0.000000  0.002079  0.008316   
252  0.000000  0.002151    0.000000  0.015054  0.000000  0.000000  0.025806   
253  0.000000  0.004762    0.000000  0.000000  0.000000  0.000000  0.016667   
254  0.000000  0.000000    0.000000  0.000000  0.000000  0.000000  0.003906   

       differ   countri    cultur  ...   straddl   

df-idf

In [17]:
# df-idf
import math
df = []
for x in tf: #針對每一個字(每個column)
    df_word = 0
    for num in tf[x]: #針對每一欄位的內容
        if num > 0: #代表有出現在app中
            df_word += 1
    df.append(df_word)
print(df)
num_column = tf.shape[0]
df_idf = [math.log(num_column/i) for i in df]
print(df_idf)

    

[1, 28, 6, 71, 43, 52, 180, 60, 27, 33, 8, 26, 113, 140, 51, 245, 178, 79, 66, 137, 55, 40, 51, 64, 80, 46, 30, 18, 26, 45, 43, 85, 155, 105, 29, 9, 10, 16, 40, 23, 170, 3, 126, 139, 43, 90, 36, 107, 9, 28, 85, 175, 61, 49, 92, 75, 12, 54, 34, 25, 61, 38, 61, 38, 2, 76, 23, 55, 5, 79, 72, 33, 134, 18, 59, 10, 41, 242, 73, 30, 160, 1, 46, 47, 14, 5, 29, 12, 100, 102, 31, 5, 75, 8, 8, 61, 83, 247, 95, 100, 84, 6, 30, 141, 61, 3, 65, 9, 35, 32, 63, 51, 45, 95, 10, 12, 104, 83, 9, 166, 52, 21, 7, 16, 19, 31, 78, 49, 5, 8, 3, 18, 12, 24, 1, 5, 52, 21, 180, 89, 97, 11, 104, 78, 91, 174, 10, 65, 16, 10, 2, 3, 1, 2, 1, 1, 106, 8, 10, 15, 1, 34, 57, 9, 10, 3, 31, 19, 36, 95, 57, 52, 6, 88, 141, 40, 12, 43, 3, 4, 8, 13, 167, 67, 7, 1, 60, 77, 36, 68, 17, 17, 10, 121, 56, 49, 93, 21, 24, 35, 18, 100, 61, 108, 14, 1, 1, 9, 16, 7, 1, 1, 8, 3, 9, 78, 61, 133, 180, 5, 133, 11, 14, 134, 72, 24, 38, 23, 39, 87, 218, 35, 137, 95, 152, 24, 156, 76, 116, 22, 189, 61, 46, 75, 101, 88, 24, 48, 68, 52, 31, 9

In [26]:
import numpy as np
num_app =  tf.shape[0]
df = (tf > 0).sum() #加總
print(df)
idf = np.log(num_app/(df+1)) #math.log，只能處理 純 Python 的數字型態（int、float）#numpy.log 專門設計來處理 向量化運算。可以直接對 NumPy 陣列 或 Pandas Series 做逐元素計算，回傳一個新的陣列或 Series
print(idf)


talkin          1
global         28
multilingu      6
commun         71
where          43
               ..
recreat         1
learnt          1
microlesson     1
oblig           1
abaenglish      1
Length: 4324, dtype: int64
talkin         4.848116
global         2.173968
multilingu     3.595353
commun         1.264597
where          1.757074
                 ...   
recreat        4.848116
learnt         4.848116
microlesson    4.848116
oblig          4.848116
abaenglish     4.848116
Length: 4324, dtype: float64


In [27]:
#用一個 DataFrame 乘以一個 Series 時，它會自動比對「欄位名稱」。
tf_idf = tf * idf
print(tf_idf)

       talkin    global  multilingu    commun     where      voic      from  \
0    0.144223  0.054722    0.024682  0.008681  0.004021  0.028759  0.005491   
1    0.000000  0.000000    0.000000  0.000000  0.000000  0.000000  0.001106   
2    0.000000  0.000000    0.000000  0.000000  0.004193  0.000000  0.000000   
3    0.000000  0.000000    0.000000  0.000000  0.000000  0.000000  0.002678   
4    0.000000  0.000000    0.000000  0.000000  0.000000  0.000000  0.002255   
..        ...       ...         ...       ...       ...       ...       ...   
250  0.000000  0.000000    0.000000  0.000000  0.000000  0.000000  0.004081   
251  0.000000  0.004520    0.000000  0.007887  0.000000  0.003266  0.002850   
252  0.000000  0.004675    0.000000  0.019037  0.000000  0.000000  0.008846   
253  0.000000  0.010352    0.000000  0.000000  0.000000  0.000000  0.005713   
254  0.000000  0.000000    0.000000  0.000000  0.000000  0.000000  0.001339   

       differ   countri    cultur  ...   straddl   

In [37]:
df_review = pd.read_csv('./data_clean_review.csv')
all_app_counts = []
for content in df_review["clean_review"]:
    counts = Counter(content.split())
    total_lengh = len(content.split())
    for k,v in counts.items():
            counts[k] = v/total_lengh
    all_app_counts.append(counts)
tf_reivew = pd.DataFrame(all_app_counts) #用 Pandas 一口氣對齊所有單字
tf_reivew = tf_reivew.fillna(0)

num_app =  tf.shape[0]

df_review = (tf_reivew > 0).sum()
idf_review = np.log(num_app/(df_review+1))
tf_idf_review = tf_reivew * idf_review


In [44]:
merge_review_result = df_review.groupby('appId')['clean_review'].apply(lambda x : ''. join(x)).reset_index()


In [45]:
print(merge_review_result)

                                                 appId  \
0                                ai.ai1min.application   
1          ai.artificial.intelligence.genai.generative   
2    ai.artificialintelligence.artificial.intellige...   
3                          ai.metaverselabs.grammargpt   
4                                    ai.ouraca.aibrary   
..                                                 ...   
254                              tinyleo.com.grammarai   
255                                      tunein.player   
256                                 tv.englishclub.b2c   
257                                 us.nobarriers.elsa   
258                                world.petr.vokabeln   

                                          clean_review  
0    ['ve', 'tri', 'lot', 'ai', 'tool', 'honestli',...  
1    ['s', 'quit', 'good']['consid', 'myself', 'hyp...  
2    ['excel', 'way', 'enhanc', 'your', 'knowledg',...  
3    ['veri', 'best', 'use']['absolut', 'love', 'ha...  
4    ['just', 'dow

In [38]:
print(tf_idf_review)

      ['best',  'struggl',  'learner',  'realli',   'help',     'u',  \
0     0.049758    0.395525    0.310567  -0.019356 -0.069229  0.24466   
1     0.000000    0.000000    0.000000  -0.000000 -0.000000  0.00000   
2     0.000000    0.115361    0.000000  -0.000000 -0.000000  0.00000   
3     0.000000    0.000000    0.000000  -0.000000 -0.000000  0.00000   
4     0.000000    0.000000    0.000000  -0.000000 -0.000000  0.00000   
...        ...         ...         ...        ...       ...      ...   
7229  0.000000    0.000000    0.000000  -0.000000 -0.000000  0.00000   
7230  0.000000    0.000000    0.000000  -0.000000 -0.000000  0.00000   
7231  0.000000    0.000000    0.000000  -0.000000 -0.000000  0.00000   
7232  0.000000    0.000000    0.000000  -0.000000 -0.000000  0.00000   
7233  0.000000    0.000000    0.000000  -0.000000 -0.000000  0.00000   

      'understand']  ['great',  'when',      'm',  ...  'gradual',  \
0          0.343681   0.000000  0.00000  0.000000  ...         0.

cosine_similarity


In [7]:
import pandas as pd
from collections import Counter
df1 = pd.read_csv('./data_metadata_clean.csv',index_col="appId")
print(df1)


                                                                   title  \
appId                                                                      
com.talkin.learn                           Talkin: Global Friends & Chat   
com.magoosh.gre.quiz.vocabulary           Vocabulary Builder - Test Prep   
com.atistudios.italk.ja                   Learn Japanese. Speak Japanese   
com.sirius                                SiriusXM: Music, Sports & News   
air.com.rosettastone.mobile.CoursePlayer  Rosetta Stone: Learn Languages   
...                                                                  ...   
com.maketabae.ielts.test.preparation       IELTS - Preparation and Tests   
com.hellotalk                                HelloTalk - Learn Languages   
com.busuu.android.enc                     Busuu: Learn & Speak Languages   
com.lms.greatlakes                        Great Learning: Online Courses   
com.abaenglish.videoclass                    ABA English - Learn English   

           

In [4]:
df2 = pd.read_csv('./data_review_merge.csv',index_col="appId")
print(df2)

                                                                                         clean_review
appId                                                                                                
ai.ai1min.application                               ve tri lot ai tool honestli most them feel eit...
ai.artificial.intelligence.genai.generative         s quit good consid myself hyper mind tend be a...
ai.artificialintelligence.artificial.intelligen...  excel way enhanc knowledg most import ai conce...
ai.metaverselabs.grammargpt                         veri best use absolut love ha help me becom be...
ai.ouraca.aibrary                                   just download keep say no internet connect whi...
...                                                                                               ...
tinyleo.com.grammarai                               wa charg servic didn t commit luckili my card ...
tunein.player                                       thi best can get do what do so

In [73]:
for x in df1.iterrows():
    print(x)

('com.talkin.learn', title                                    Talkin: Global Friends & Chat
description          Talkin is a global multilingual community wher...
realInstalls                                                   4235634
score                                                             3.47
ratings                                                         6705.0
categories                        [{'name': 'Social', 'id': 'SOCIAL'}]
days_since_update                                                  6.0
survive                                                              1
clean_description    talkin global multilingu commun where voic fro...
Name: com.talkin.learn, dtype: object)
('com.magoosh.gre.quiz.vocabulary', title                                   Vocabulary Builder - Test Prep
description          <b>★ Over 5 MILLION students have used Magoosh...
realInstalls                                                  10451121
score                                               

In [10]:
import numpy as np
def build_tf_idf(df_clean,column): #建立tf-idf 時就應該有index = appId
    all_app_counts = {}

    for appId,row in df_clean.iterrows():
        content = row[column]
        counts = Counter(content.split()) #每個 App 算好它的詞頻小字典
        total_lengh = len(content.split())
        for k,v in counts.items():
            counts[k] = v/total_lengh
        
        all_app_counts[appId] = counts

    tf = pd.DataFrame(all_app_counts) #用 Pandas 一口氣對齊所有單字
    tf = tf.fillna(0)


    num_app =  tf.shape[1] #計算有幾欄(有幾個app)
    df = (tf > 0).sum() #加總
    idf = np.log(num_app/(df+1))

    tf_idf = tf * idf
    df_idf__transposed = tf_idf.T
    return df_idf__transposed

In [21]:
df_idf_1 = build_tf_idf(df1,'clean_description')
print(df_idf_1)

                                            talkin    global  multilingu  \
com.talkin.learn                          0.005651  0.004782    0.001304   
com.magoosh.gre.quiz.vocabulary           0.000000  0.000000    0.000000   
com.atistudios.italk.ja                   0.000000  0.000000    0.000000   
com.sirius                               -0.000000 -0.000000   -0.000000   
air.com.rosettastone.mobile.CoursePlayer  0.000000  0.000000    0.000000   
...                                            ...       ...         ...   
com.maketabae.ielts.test.preparation      0.000000  0.000000    0.000000   
com.hellotalk                            -0.000000 -0.000027   -0.000000   
com.busuu.android.enc                     0.000000  0.000940    0.000000   
com.lms.greatlakes                        0.000000  0.001134    0.000000   
com.abaenglish.videoclass                 0.000000  0.000000    0.000000   

                                            commun     where      voic  \
com.talkin.le

In [22]:
df_idf_2 = build_tf_idf(df2,'clean_review')
print(df_idf_2)

                                                          ve       tri  \
ai.ai1min.application                              -0.001978 -0.000495   
ai.artificial.intelligence.genai.generative         0.000629  0.000000   
ai.artificialintelligence.artificial.intelligen...  0.000569  0.001708   
ai.metaverselabs.grammargpt                         0.000000  0.000844   
ai.ouraca.aibrary                                   0.000000  0.000000   
...                                                      ...       ...   
tinyleo.com.grammarai                               0.000000  0.000000   
tunein.player                                       0.000565  0.000000   
tv.englishclub.b2c                                  0.001763  0.001763   
us.nobarriers.elsa                                  0.000000  0.001646   
world.petr.vokabeln                                -0.000454 -0.001818   

                                                         lot        ai  \
ai.ai1min.application                

cosine
先對齊 appId，再對齊 term，最後才算 cosine

In [30]:
# 先對齊「appId」
## 找出共同索引
common_app  = df_idf_1.index.intersection(df_idf_2.index)
print(common_app)

X = df_idf_1.loc[common_app] # .loc 可用來篩選row 不用寫for 迴圈
print(X)
Y = df_idf_2.loc[common_app]
print(Y)

Index(['com.talkin.learn', 'com.magoosh.gre.quiz.vocabulary',
       'com.atistudios.italk.ja', 'com.sirius',
       'air.com.rosettastone.mobile.CoursePlayer', 'com.radio.pocketfm',
       'app.helloado.contenu', 'com.dzinemedia.grammar.checker.spell',
       'ru.poas.frenchwords', 'com.google.android.apps.classroom',
       ...
       'com.moymer.falou', 'org.pbskids.gamesapp', 'com.dreamai.synthiai',
       'com.masterenglishprogram', 'com.prodigygame.prodigy',
       'com.mixlr.android', 'com.hellotalk', 'com.busuu.android.enc',
       'com.lms.greatlakes', 'com.abaenglish.videoclass'],
      dtype='object', length=249)
                                            talkin    global  multilingu  \
com.talkin.learn                          0.005651  0.004782    0.001304   
com.magoosh.gre.quiz.vocabulary           0.000000  0.000000    0.000000   
com.atistudios.italk.ja                   0.000000  0.000000    0.000000   
com.sirius                               -0.000000 -0.000000   -

In [32]:
# 再對齊「詞彙空間」
X_aligned, Y_aligned = X.align(Y, join="outer", axis=1, fill_value=0)
print(X_aligned)

                                          aa  aacha  aadil  aap  aapka  \
com.talkin.learn                           0      0      0    0      0   
com.magoosh.gre.quiz.vocabulary            0      0      0    0      0   
com.atistudios.italk.ja                    0      0      0    0      0   
com.sirius                                 0      0      0    0      0   
air.com.rosettastone.mobile.CoursePlayer   0      0      0    0      0   
...                                       ..    ...    ...  ...    ...   
com.mixlr.android                          0      0      0    0      0   
com.hellotalk                              0      0      0    0      0   
com.busuu.android.enc                      0      0      0    0      0   
com.lms.greatlakes                         0      0      0    0      0   
com.abaenglish.videoclass                  0      0      0    0      0   

                                          aarigato  aati  ab       aba  \
com.talkin.learn                     

In [33]:
print(Y_aligned)

                                           aa  aacha  aadil  aap  aapka  \
com.talkin.learn                         -0.0   -0.0   -0.0 -0.0   -0.0   
com.magoosh.gre.quiz.vocabulary           0.0    0.0    0.0  0.0    0.0   
com.atistudios.italk.ja                   0.0    0.0    0.0  0.0    0.0   
com.sirius                               -0.0   -0.0   -0.0 -0.0   -0.0   
air.com.rosettastone.mobile.CoursePlayer -0.0   -0.0   -0.0 -0.0   -0.0   
...                                       ...    ...    ...  ...    ...   
com.mixlr.android                         0.0    0.0    0.0  0.0    0.0   
com.hellotalk                            -0.0   -0.0   -0.0 -0.0   -0.0   
com.busuu.android.enc                     0.0    0.0    0.0  0.0    0.0   
com.lms.greatlakes                        0.0    0.0    0.0  0.0    0.0   
com.abaenglish.videoclass                 0.0    0.0    0.0  0.0    0.0   

                                          aarigato  aati   ab       aba  \
com.talkin.learn        

In [36]:
X_aligned * Y_aligned

,aa,aacha,aadil,aap,aapka,aarigato,aati,ab,aba,abaenglish,...,zombi,zone,zoom,zor,zu,zulu,zum,zurich,zx,zyka
com.talkin.learn,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000000,0.0,...,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,-0.0
com.magoosh.gre.quiz.vocabulary,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
com.atistudios.italk.ja,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
com.sirius,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.000000,-0.0,...,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
air.com.rosettastone.mobile.CoursePlayer,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000000,0.0,...,0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
com.mixlr.android,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
com.hellotalk,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.000000,-0.0,...,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
com.busuu.android.enc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
com.lms.greatlakes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


clustering

In [80]:
# 先把資料load 進來
df1 = pd.read_csv('metadata_add_feature.csv',index_col='appId')
df2 = pd.read_csv('review_add_feature.csv',index_col='appId')


In [83]:
#先對齊appId
common_appId = df1.index.intersection(df2.index)
df1_common_appId = df1.loc[common_appId] #取出共同的index 
df2_common_appId = df2.loc[common_appId]
print(df2_common_appId)


                                                                     clean_review  \
appId                                                                               
com.ai.skillium                 cool good awesom super applic realli good lear...   
es.monkimun.lingokids           love hello thought thi wa free game but can t ...   
com.talkin.learn                good m not abl login into thi becaus m use and...   
com.duolingo.literacy           love thi engag fun educ free no ad work great ...   
com.magoosh.flashcards.gre      thi applic brim divers recondit vocab wa genui...   
...                                                                           ...   
ru.poas.frenchwords             lot info pronunci vocab awesom start studi fre...   
com.milinix.englishgrammartest  good exercis good but there are so mani mistak...   
com.lms.greatlakes              teacher are good cost econom cours upskil some...   
com.hellochinese                realli like thi becaus s easi use

In [ ]:
#把需要的欄位分別取出來
df1_filter_features = df1_common_appId[['text_lengh','unique_word_count','lexical_diversity','avg_word_length']]
df2_filter_features = df2_common_appId[['text_lengh','unique_word_count','lexical_diversity','avg_word_length']]
df1_filter_features.columns = ['desc_text_lengh','desc_unique_word_count','desc_lexical_diversity','desc_avg_word_length']
df2_filter_features.columns = ['rev_text_lengh','rev_unique_word_count','rev_lexical_diversity','rev_avg_word_length']

#正式合併
result = pd.concat([df1_filter_features, df2_filter_features], axis=1) #合併
result = result.fillna(0)#有沒有 NaN
#print(result)

#再合併cosine_similary
df3 = pd.read_csv('cosine_similarity_result.csv',index_col=0)
##再對齊一次appId
common_appId = result.index.intersection(df3.index)
result_filter = result.loc[common_appId] #取出共同的index 
df3_filter = df3.loc[common_appId]
final = pd.concat([result_filter,df3_filter],axis=1)



OSError: Cannot save file into a non-existent directory: 'google-play-app-survival\notebooks'

In [5]:
import pandas as pd
df = pd.read_csv('feaures_labels.csv',index_col=0)
a = df.groupby('cluster').mean()
print(a)
print(type(a))

         desc_text_lengh  desc_unique_word_count  desc_lexical_diversity  \
cluster                                                                    
0               0.648287                0.572082                0.257215   
1               0.419127                0.483559                0.514948   
2               0.747730                0.660121                0.235695   

         desc_avg_word_length  rev_text_lengh  rev_unique_word_count  \
cluster                                                                
0                    0.370142        0.180678               0.261896   
1                    0.412503        0.227625               0.323056   
2                    0.371714        0.393384               0.492845   

         rev_lexical_diversity  rev_avg_word_length  cosine_similarity  
cluster                                                                 
0                     0.425345             0.477934           0.375598  
1                     0.349206         